In [3]:
import random  
random.seed(42)  

import numpy as np  
import matplotlib.pyplot as plt 
import seaborn as sns  

from collections import Counter  

import torch  
import torch.nn as nn  
import torch.nn.functional as F  
import torch.optim as optim  

from torch.nn import BatchNorm1d  
from torch_geometric.data import Data  
from torch_geometric.nn import GCNConv, GATConv  
from torch_geometric.transforms import ToUndirected  
from torch_geometric.transforms import ToUndirected
from torch_geometric.utils import add_self_loops  

from sklearn.metrics import roc_curve, auc, confusion_matrix  
from sklearn.preprocessing import MinMaxScaler, label_binarize

import os
import pickle
import uproot

In [4]:
file = uproot.open('/storage/mxg1065/MyxAODAnalysis_super3D.outputs.root')
print(file.keys())

['analysis;1']


In [5]:
tree = file['analysis;1']
branches = tree.arrays()
print(tree.keys()) # Variables per event

['RunNumber', 'EventNumber', 'cell_eta', 'cell_phi', 'cell_x', 'cell_y', 'cell_z', 'cell_subCalo', 'cell_sampling', 'cell_size', 'cell_hashID', 'neighbor', 'seedCell_id', 'cell_e', 'cell_noiseSigma', 'cell_SNR', 'cell_time', 'cell_weight', 'cell_truth', 'cell_truth_indices', 'cell_shared_indices', 'cell_cluster_index', 'cluster_to_cell_indices', 'cluster_to_cell_weights', 'cell_to_cluster_e', 'cell_to_cluster_eta', 'cell_to_cluster_phi', 'cluster_eta', 'cluster_phi', 'cluster_e', 'cellsNo_cluster', 'clustersNo_event', 'jetEnergyWtdTimeAve', 'jetEta', 'jetPhi', 'jetE', 'jetPt', 'jetNumberPerEvent', 'cellIndices_per_jet']


### Prepairing the Data Dictionary

In [6]:
# 100 events and 187652 cells
# Arrays containing information about the energy, noise, snr, 
cell_e = np.array(branches['cell_e'])
cell_noise = np.array(branches['cell_noiseSigma'])
cell_snr = np.array(branches['cell_SNR'])
cell_eta = np.array(branches['cell_eta'])
cell_phi = np.array(branches['cell_phi'])

# Represents the index of the cluster that each cell corresponds to. If the index
# is 0, that means that the given cell does not belong to a cluster.
cell_to_cluster_index = np.array(branches['cell_cluster_index'])

# For each entry, contains the IDs of cells neighboring a given cell
neighbor = branches['neighbor']


In [7]:
num_of_events = len(cell_snr)
num_of_cells = len(cell_snr[0])
print("Number of Events:", num_of_events)
print("Number of Cells:", num_of_cells)

Number of Events: 100
Number of Cells: 187652


In [8]:
# We use the data arrays to crete a data dictionary, where each entry corresponds
# to the data of a given event; we scale this data.
data = {}

for i in range(num_of_events):
    data[f'data_{i}'] = np.concatenate((np.expand_dims(cell_snr[i], axis=1),
                                        np.expand_dims(cell_e[i], axis=1),
                                        np.expand_dims(cell_noise[i], axis=1),
                                        np.expand_dims(cell_eta[i], axis=1),
                                        np.expand_dims(cell_phi[i], axis=1)), axis=1)
    
# We combine the data into one array and apply the MinMaxScaler
combined_data = np.vstack([data[key] for key in data])
scaler = MinMaxScaler()
scaled_combined_data = scaler.fit_transform(combined_data)

# The scaled data is split to have the save structure as the original data dict
scaled_data = {}
start_idx = 0
for i in range(num_of_events):
    end_idx = start_idx + data[f"data_{i}"].shape[0]
    scaled_data[f"data_{i}"] = scaled_combined_data[start_idx:end_idx]
    start_idx = end_idx

print(scaled_data["data_0"])
print(scaled_data["data_0"].shape)

[[0.02640459 0.24943821 0.09700817 0.23466232 0.5085498 ]
 [0.02567646 0.24627675 0.09700815 0.23466876 0.52418756]
 [0.02508186 0.24369499 0.09700817 0.23467347 0.5398244 ]
 ...
 [0.02632877 0.24791998 0.03177016 0.62017125 0.4921177 ]
 [0.02705116 0.2511391  0.07512318 0.6461531  0.4921177 ]
 [0.02638626 0.24820389 0.04149057 0.6731673  0.4921177 ]]
(187652, 5)


### Preparing Neighbor Pairs

In [9]:
# The IDs of the broken cells (those with zero noise) are collected
broken_cells = []

for i in range(num_of_events):
    cells = np.argwhere(cell_noise[i]==0)
    broken_cells = np.squeeze(cells)

print(broken_cells)

[186986 187352]


In [10]:
# Since the values associated with neighbor[0] and neighbor[1] are all equal
# we will just work with neighbor[0] to simplify our calculations
neighbor = neighbor[0]

In [11]:
# We loop through the neighbor awkward array and remove the IDs associated
# with the broken cells.  Loops through all cells in the neighbor list. If the loop 
# reaches the cell numbers 186986 or 187352, loop skips over these inoperative cells. 
# The final list contains tuples (i,j) where i is the cell ID in question and the 
# js are the neighboring cell IDs
neighbor_pairs_list = []

for i in range(num_of_cells):
    if i in broken_cells:
        continue
    for j in neighbor[i]:
        if j in broken_cells:
            continue
        neighbor_pairs_list.append((i, int(j)))

In [12]:
# This code checks to see if the broken cells were removed
found_broken_cells = []

for pair in neighbor_pairs_list:
    # Loop through each cell in pair
    for cell in pair:
        # If the cell is broken, appends to list
        if cell in broken_cells:
            found_broken_cells.append(cell)

if found_broken_cells:
    print("Error: Broken cells are still present in neighbor pairs.")
else:
    print("Successfully excluded broken cells.")

Successfully excluded broken cells.


In [13]:
# These functions remove permutation variants
def canonical_form(t):
    return tuple(sorted(t))

def remove_permutation_variants(tuple_list):
    unique_tuples = set(canonical_form(t) for t in tuple_list)
    return [tuple(sorted(t)) for t in unique_tuples]

neighbor_pairs_list = np.array(remove_permutation_variants(neighbor_pairs_list))
print(neighbor_pairs_list)
print(neighbor_pairs_list.shape)

[[ 90345 119588]
 [  4388  17680]
 [ 39760  39825]
 ...
 [159757 168717]
 [ 62911  78974]
 [135353 135609]]
(1250242, 2)


### Creating Labels for the Neighbor Pairs

For a given pair of cells and the IDs of the clusters that they belong to (i, j), if
1. i=j and both are nonzero, then both cells are part of the same cluster. 
    * We call these True-True pairs and label them with 1
2. i=j and both are zero, then both cells are not part of any cluster. 
    * We call these Lone-Lone pairs and label them with 0
3. i is nonzero and j=0, then cell i is part of a cluster while cell j is not. 
    * We call these Cluster-Lone pairs and label them with 2
4. i=0 and j is nonzero, then cell i is not part of a cluste while cell j is. 
    * We call these Lone-Cluster pairs and label them with 3
5. i is not the same as j and both are nonzero, then both cells are part of different clusters. 
    * We call these Cluster-Cluster pairs and label them with 4

In [14]:
# Initialize array for labels
labels_for_neighbor_pairs = np.zeros((num_of_events, len(neighbor_pairs_list)), dtype=int)

# Extracting the individual cells within a cell pair
cell_0 = cell_to_cluster_index[:, neighbor_pairs_list[:, 0]]
cell_1 = cell_to_cluster_index[:, neighbor_pairs_list[:, 1]]

# Computing labels using vectorized operations
same_cluster = cell_0 == cell_1 
both_nonzero = (cell_0 != 0) & (cell_1 != 0)

# Lone-Lone (0)
labels_for_neighbor_pairs[same_cluster & (cell_0 == 0)] = 0

# True-True (1)
labels_for_neighbor_pairs[same_cluster & (cell_0 != 0)] = 1

# Cluster-Cluster (4)
labels_for_neighbor_pairs[~same_cluster & both_nonzero] = 4

# Lone-Cluster (3)
labels_for_neighbor_pairs[~same_cluster & (cell_0 == 0) & (cell_1 != 0)] = 3

# Cluster-Lone (2)
labels_for_neighbor_pairs[~same_cluster & (cell_0 != 0) & (cell_1 == 0)] = 2

print(labels_for_neighbor_pairs.shape)
print(labels_for_neighbor_pairs)
print(np.unique(labels_for_neighbor_pairs[0]))

(100, 1250242)
[[0 0 0 ... 0 0 2]
 [3 3 0 ... 0 0 2]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 2 0 0]]
[0 1 2 3 4]


### Preparing the Data for Multi-Class Classification

In [15]:
# Here we collect the indices of the neighbor pairs by the pair type
indices_for_tt_pairs = []  # Label 1
indices_for_ll_pairs = []  # Label 0
indices_for_cl_pairs = []  # Label 2
indices_for_lc_pairs = []  # Label 3
indices_for_cc_pairs = []  # Label 4

for i in range(num_of_events):
    indices_for_tt_pairs.append(list(np.where(labels_for_neighbor_pairs[i] == 1)[0]))
    indices_for_ll_pairs.append(list(np.where(labels_for_neighbor_pairs[i] == 0)[0]))
    indices_for_cl_pairs.append(list(np.where(labels_for_neighbor_pairs[i] == 2)[0]))
    indices_for_lc_pairs.append(list(np.where(labels_for_neighbor_pairs[i] == 3)[0]))
    indices_for_cc_pairs.append(list(np.where(labels_for_neighbor_pairs[i] == 4)[0]))

In [16]:
# Here we collect the number of each pair type across the events
number_of_tt_pairs = [len(indices_for_tt_pairs[i])for i in range(num_of_events)]
number_of_ll_pairs = [len(indices_for_ll_pairs[i])for i in range(num_of_events)]
number_of_cl_pairs = [len(indices_for_cl_pairs[i])for i in range(num_of_events)]
number_of_lc_pairs = [len(indices_for_lc_pairs[i])for i in range(num_of_events)]
number_of_cc_pairs = [len(indices_for_cc_pairs[i])for i in range(num_of_events)]

In [17]:
# Here we perform a 80-20 split on the indices of neighbor pairs
training_indices_tt = indices_for_tt_pairs[:80]
training_indices_ll = indices_for_ll_pairs[:80]
training_indices_cl = indices_for_cl_pairs[:80]
training_indices_lc = indices_for_lc_pairs[:80]
training_indices_cc = indices_for_cc_pairs[:80]

testing_indices_tt = indices_for_tt_pairs[80:]
testing_indices_ll = indices_for_ll_pairs[80:]
testing_indices_cl = indices_for_cl_pairs[80:]
testing_indices_lc = indices_for_lc_pairs[80:]
testing_indices_cc = indices_for_cc_pairs[80:]

# Here we perform a 80-20 split on the number of neighbor pairs
training_num_tt = number_of_tt_pairs[:80]
training_num_ll = number_of_ll_pairs[:80]
training_num_cl = number_of_cl_pairs[:80]
training_num_lc = number_of_lc_pairs[:80]
training_num_cc = number_of_cc_pairs[:80]

testing_num_tt = number_of_tt_pairs[80:]
testing_num_ll = number_of_ll_pairs[80:]
testing_num_cl = number_of_cl_pairs[80:]
testing_num_lc = number_of_lc_pairs[80:]
testing_num_cc = number_of_cc_pairs[80:]

In [18]:
# We check the minimum number of each pair type across the events. When we
# randomly sample from the indices, if our sample is greater than the minimum
# numbers, then we will run into errors
print("Minimum number of pairs for training:")
print("True-True", min(training_num_tt))
print("Lone-Lone", min(training_num_ll))
print("Cluster-Lone", min(training_num_cl))
print("Lone-Cluster", min(training_num_lc))
print("Cluster-Cluster", min(training_num_cc))
print('\nMinimum number of pairs for testing:')
print("True-True", min(testing_num_tt))
print("Lone-Lone", min(testing_num_ll))
print("Cluster-Lone", min(testing_num_cl))
print("Lone-Cluster", min(testing_num_lc))
print("Cluster-Cluster", min(testing_num_cc))

Minimum number of pairs for training:
True-True 45600
Lone-Lone 926119
Cluster-Lone 41654
Lone-Cluster 45689
Cluster-Cluster 3334

Minimum number of pairs for testing:
True-True 51518
Lone-Lone 906630
Cluster-Lone 44444
Lone-Cluster 48069
Cluster-Cluster 4936


### Making training and testing sets

In [19]:
big_sample_num = 40000
small_sample_num = 3000

In [20]:
train_indices_tt_pairs = np.array([random.sample(row, big_sample_num) for row in training_indices_tt])
train_indices_ll_pairs = np.array([random.sample(row, big_sample_num) for row in training_indices_ll])
train_indices_cl_pairs = np.array([random.sample(row, big_sample_num) for row in training_indices_cl])
train_indices_lc_pairs = np.array([random.sample(row, big_sample_num) for row in training_indices_lc])
train_indices_cc_pairs = np.array([random.sample(row, small_sample_num) for row in training_indices_cc])
train_indices_bkg_pairs = np.concatenate([train_indices_ll_pairs,
                                          train_indices_cl_pairs,
                                          train_indices_lc_pairs,
                                          train_indices_cc_pairs], axis=1)

test_indices_tt_pairs = np.array([random.sample(row, big_sample_num) for row in testing_indices_tt])
test_indices_ll_pairs = np.array([random.sample(row, big_sample_num) for row in testing_indices_ll])
test_indices_cl_pairs = np.array([random.sample(row, big_sample_num) for row in testing_indices_cl])
test_indices_lc_pairs = np.array([random.sample(row, big_sample_num) for row in testing_indices_lc])
test_indices_cc_pairs = np.array([random.sample(row, small_sample_num) for row in testing_indices_cc])
test_indices_bkg_pairs = np.concatenate([test_indices_ll_pairs,
                                         test_indices_cl_pairs,
                                         test_indices_lc_pairs,
                                         test_indices_cc_pairs], axis=1)

print(train_indices_tt_pairs.shape)
print(train_indices_bkg_pairs.shape)
print(test_indices_tt_pairs.shape)
print(test_indices_bkg_pairs.shape)

(80, 40000)
(80, 123000)
(20, 40000)
(20, 123000)


In [21]:
total_training_indices = np.concatenate((train_indices_tt_pairs, train_indices_bkg_pairs), axis=1)
total_testing_indices = np.concatenate((test_indices_tt_pairs, test_indices_bkg_pairs), axis=1)
print(total_training_indices.shape)
print(total_testing_indices.shape)

(80, 163000)
(20, 163000)


### Randomizing the indicies and creating labels for training and testing

In [22]:
# Creating training labels
labels_tt_train = np.ones((80, big_sample_num), dtype=int)  # True-True
labels_ll_train = np.zeros((80, big_sample_num), dtype=int)  # Lone-Lone
labels_cl_train = np.ones((80, big_sample_num), dtype=int)*2  # Cluster-Lone
labels_lc_train = np.ones((80, big_sample_num), dtype=int)*3  # Lone-Cluster
labels_cc_train = np.ones((80, small_sample_num), dtype=int)*4  # Cluster-Cluster
labels_bkg_train = np.concatenate((labels_ll_train, labels_cl_train, labels_lc_train, labels_cc_train), axis=1)
labels_training = np.concatenate((labels_tt_train, labels_bkg_train), axis=1)

# Creating testing labels
labels_tt_test = np.ones((20, big_sample_num), dtype=int)  # True-True
labels_ll_test = np.zeros((20, big_sample_num), dtype=int)  # Lone-Lone
labels_cl_test = np.ones((20, big_sample_num), dtype=int)*2  # Cluster-Lone
labels_lc_test = np.ones((20, big_sample_num), dtype=int)*3  # Lone-Cluster
labels_cc_test = np.ones((20, small_sample_num), dtype=int)*4  # Cluster-Cluster
labels_bkg_test = np.concatenate((labels_ll_test, labels_cl_test, labels_lc_test, labels_cc_test), axis=1)
labels_testing = np.concatenate((labels_tt_test, labels_bkg_test), axis=1)

# Printing the shapes of the final training and testing labels
print(labels_training.shape)
print(labels_testing.shape)

(80, 163000)
(20, 163000)


In [23]:
# Here we randomize the training and testing information. But to do this, while keeping
# the same permutation for both indicies and labels, we use np.random.permutation

# Randomizing training data via iteration over the rows
for i in range(total_training_indices.shape[0]):
    perm = np.random.permutation(total_training_indices.shape[1])
    total_training_indices[i] = total_training_indices[i, perm]
    labels_training[i] = labels_training[i, perm]

# Randomizing testing data via iteration over the rows
for i in range(total_testing_indices.shape[0]):
    perm = np.random.permutation(total_testing_indices.shape[1])
    total_testing_indices[i] = total_testing_indices[i, perm]
    labels_testing[i] = labels_testing[i, perm]

print(total_training_indices.shape)
print(labels_training.shape)
print(total_testing_indices.shape)
print(labels_testing.shape)

(80, 163000)
(80, 163000)
(20, 163000)
(20, 163000)


In [24]:
# Arranging the neighbor pairs with the training indices
total_train_neighbor_random = []
for i in range(len(labels_training)):
    total_train_neighbor_random.append(neighbor_pairs_list[total_training_indices[i]])
total_train_neighbor_random = np.array(total_train_neighbor_random)
print(total_train_neighbor_random.shape)

# Arranging the neighbor pairs with the testing indices
total_test_neighbor_random = []
for i in range(len(labels_testing)):
    total_test_neighbor_random.append(neighbor_pairs_list[total_training_indices[i]])
total_test_neighbor_random = np.array(total_test_neighbor_random)
print(total_test_neighbor_random.shape)

(80, 163000, 2)
(20, 163000, 2)


In [25]:
def createArray(input_data, num_of_data, is_source, is_bi_directional):
    # Initialize an empty list to store the output data
    data = []

    # Loop through each set of data in input_data
    for i in range(num_of_data):
        _data = []

        # Loop through each pair of data in the current data set
        for pair in input_data[i]:

            # Process data depending on is_bi_directional flag
            if is_bi_directional:
                # If is_source is True, append both elements in original order
                if is_source:
                    _data.append(pair[0])
                    _data.append(pair[1])
                else:
                    # If is_source is False, append elements in reversed order
                    _data.append(pair[1])
                    _data.append(pair[0])
            else:
                # If is_bi_directional is False, append only one element depending on is_source flag
                if is_source:
                    _data.append(pair[0])
                else:
                    _data.append(pair[1])

        # Add the processed data set to the output list
        data.append(_data)

    # Return the final processed list of data
    data = np.array(data)
    return data

In [26]:
# Creating bi-/uni-directional arrays for processing downstream

# Training arrays
train_edge_source_bi = createArray(total_train_neighbor_random, 80, True, True)
train_edge_dest_bi = createArray(total_train_neighbor_random, 80, False, True)
train_edge_source_uni = createArray(total_train_neighbor_random, 80, True, False)
train_edge_dest_uni = createArray(total_train_neighbor_random, 80, False, False)

# Testing arrays
test_edge_source_bi = createArray(total_test_neighbor_random, 20, True, True)
test_edge_dest_bi = createArray(total_test_neighbor_random, 20, False, True)
test_edge_source_uni = createArray(total_test_neighbor_random, 20, True, False)
test_edge_dest_uni = createArray(total_test_neighbor_random, 20, False, False)

# Printing the shapes
print(train_edge_source_bi.shape)
print(train_edge_dest_bi.shape)
print(train_edge_source_uni.shape)
print(train_edge_dest_uni.shape)
print(test_edge_source_bi.shape)
print(test_edge_dest_bi.shape)
print(test_edge_source_uni.shape)
print(test_edge_dest_uni.shape)

(80, 326000)
(80, 326000)
(80, 163000)
(80, 163000)
(20, 326000)
(20, 326000)
(20, 163000)
(20, 163000)


In [27]:
# Filtering and creating neighbor pairs sorted by the indices of each neighbor type
test_neighbor_pairs_tt = []
test_neighbor_pairs_ll = []
test_neighbor_pairs_cl = []
test_neighbor_pairs_lc = []
test_neighbor_pairs_cc = []

for i in range(len(labels_testing)):
    test_neighbor_pairs_tt.append(neighbor_pairs_list[test_indices_tt_pairs[i]])
    test_neighbor_pairs_ll.append(neighbor_pairs_list[test_indices_ll_pairs[i]])
    test_neighbor_pairs_cl.append(neighbor_pairs_list[test_indices_cl_pairs[i]])
    test_neighbor_pairs_lc.append(neighbor_pairs_list[test_indices_lc_pairs[i]])
    test_neighbor_pairs_cc.append(neighbor_pairs_list[test_indices_cc_pairs[i]])

test_neighbor_pairs_tt = np.array(test_neighbor_pairs_tt)
test_neighbor_pairs_ll = np.array(test_neighbor_pairs_ll)
test_neighbor_pairs_cl = np.array(test_neighbor_pairs_cl)
test_neighbor_pairs_lc = np.array(test_neighbor_pairs_lc)
test_neighbor_pairs_cc = np.array(test_neighbor_pairs_cc)

In [28]:
test_edge_tt_source_bi = createArray(test_neighbor_pairs_tt, 20, True, True)
test_edge_tt_dest_bi = createArray(test_neighbor_pairs_tt, 20, False, True)
test_edge_tt_source_uni = createArray(test_neighbor_pairs_tt, 20, True, False)
test_edge_tt_dest_uni = createArray(test_neighbor_pairs_tt, 20, False, False)

test_edge_ll_source_bi = createArray(test_neighbor_pairs_ll, 20, True, True)
test_edge_ll_dest_bi = createArray(test_neighbor_pairs_ll, 20, False, True)
test_edge_ll_source_uni = createArray(test_neighbor_pairs_ll, 20, True, False)
test_edge_ll_dest_uni = createArray(test_neighbor_pairs_ll, 20, False, False)

test_edge_cl_source_bi = createArray(test_neighbor_pairs_cl, 20, True, True)
test_edge_cl_dest_bi = createArray(test_neighbor_pairs_cl, 20, False, True)
test_edge_cl_source_uni = createArray(test_neighbor_pairs_cl, 20, True, False)
test_edge_cl_dest_uni = createArray(test_neighbor_pairs_cl, 20, False, False)

test_edge_lc_source_bi = createArray(test_neighbor_pairs_lc, 20, True, True)
test_edge_lc_dest_bi = createArray(test_neighbor_pairs_lc, 20, False, True)
test_edge_lc_source_uni = createArray(test_neighbor_pairs_lc, 20, True, False)
test_edge_lc_dest_uni = createArray(test_neighbor_pairs_lc, 20, False, False)

test_edge_cc_source_bi = createArray(test_neighbor_pairs_cc, 20, True, True)
test_edge_cc_dest_bi = createArray(test_neighbor_pairs_cc, 20, False, True)
test_edge_cc_source_uni = createArray(test_neighbor_pairs_cc, 20, True, False)
test_edge_cc_dest_uni = createArray(test_neighbor_pairs_cc, 20, False, False)


# Print shapes
print(test_edge_tt_source_bi.shape)
print(test_edge_tt_dest_bi.shape)
print(test_edge_tt_source_uni.shape)
print(test_edge_tt_dest_uni.shape)

(20, 80000)
(20, 80000)
(20, 40000)
(20, 40000)


## Making the features array for training and testing

In [29]:
# Here we create a rearranged dictionary from the scaled data dictionary
keys = list(scaled_data.keys())
values = list(scaled_data.values())
features_dict = dict(zip(keys, values))

# Here these features are split into training and testing sets
features_training = np.concatenate([value for key, value in list(features_dict.items())[:80]])
features_testing = np.concatenate([value for key, value in list(features_dict.items())[80:]])

In [30]:
print(features_training.shape)
print(features_testing.shape)

(15012160, 5)
(3753040, 5)


In [31]:
features_training = features_training.reshape(80, 187652, 5)
features_testing = features_testing.reshape(20, 187652, 5)

# Creation of the NN Model

In [32]:
# Make the scaled features into a torch tensor (inputs)
x_train = torch.tensor(features_training, dtype=torch.float)
x_test = torch.tensor(features_testing, dtype=torch.float)

In [33]:
# Here the correct dimension permutations are applied for the model
def make_edge_index_tensor(source, dest):
    edge_index = np.stack([np.array(source), np.array(dest)], axis=0)  # Stack into a single NumPy array
    return torch.tensor(edge_index, dtype=torch.long).permute(1, 0, 2)

# Training set (Bi-directional and Uni-directional)
train_edge_indices_bi = make_edge_index_tensor(train_edge_source_bi, train_edge_dest_bi)
train_edge_indices_uni = make_edge_index_tensor(train_edge_source_uni, train_edge_dest_uni)

print(train_edge_indices_bi.shape)
print(train_edge_indices_uni.shape)


torch.Size([80, 2, 326000])
torch.Size([80, 2, 163000])


In [34]:
edge_index_data = {
    "tt_bi": (test_edge_tt_source_bi, test_edge_tt_dest_bi),
    "tt_uni": (test_edge_tt_source_uni, test_edge_tt_dest_uni),
    "ll_bi": (test_edge_ll_source_bi, test_edge_ll_dest_bi),
    "ll_uni": (test_edge_ll_source_uni, test_edge_ll_dest_uni),
    "cl_bi": (test_edge_cl_source_bi, test_edge_cl_dest_bi),
    "cl_uni": (test_edge_cl_source_uni, test_edge_cl_dest_uni),
    "lc_bi": (test_edge_lc_source_bi, test_edge_lc_dest_bi),
    "lc_uni": (test_edge_lc_source_uni, test_edge_lc_dest_uni),
    "cc_bi": (test_edge_cc_source_bi, test_edge_cc_dest_bi),
    "cc_uni": (test_edge_cc_source_uni, test_edge_cc_dest_uni),
}

# Create and permute tensors for all edge types
edge_indices = {key: make_edge_index_tensor(sources, dests) for key, (sources, dests) in edge_index_data.items()}

In [35]:
y_train = np.expand_dims(labels_training, axis=1)
y_train = torch.tensor(y_train)
y_test = np.expand_dims(labels_testing, axis=1)
y_test = torch.tensor(y_test)

### Creation of custom data lists, collate functions, and data loaders

In [36]:
# Create a class that inherents from the torch.utils.data.Dataset class
# The pytorch class is abstract, meaning we need to define certain methods
# like __len__() and __getitem__()
class custom_dataset(torch.utils.data.Dataset):
    # Class constructor that takes in data list and
    # stores it as an instance, making it avaliable
    # to other methods in the class
    def __init__(self, data_list):
        self.data_list = data_list

    # Method return length of data set
    def __len__(self):
        return len(self.data_list)

    # Method returns data point at index idx
    def __getitem__(self, idx):
        return self.data_list[idx]

# Used to handle batch loading, shuffling, and parallel loading during
# training and testing in the ML pipeline

In [37]:
# Create a list with information regarding a homogenous graph (a graph
# where all nodes represent instances of the same type [cells in the
# detector] and all edges represent relations of the same type [connections
# between cells])
def create_data_list(bi_edge_indices, uni_edge_indices, x, y):
    data_list = []
    for i in range(len(bi_edge_indices)):
        # Create the feature matrix
        x_mat = x[i]
        # Create graph connectivity matrix
        edge_index = bi_edge_indices[i]
        edge_index, _ = add_self_loops(edge_index)

        # Convert y[i] to a PyTorch tensor
        y_tensor = torch.tensor(y[i], dtype=torch.long) if not isinstance(
            y[i], torch.Tensor) else y[i]

        # Create the data object describing a homogeneous graph
        data = Data(x=x_mat, edge_index=edge_index,
                    edge_index_out=uni_edge_indices[i], y=y_tensor)
        data = ToUndirected()(data)
        data_list.append(data)
    return data_list


def collate_data(data_list):
    return ([data.x for data in data_list],
            [data.edge_index for data in data_list],
            [data.edge_index_out for data in data_list],
            torch.cat([data.y for data in data_list], dim=0))

In [38]:
# Create the data lists for all edge types and categories

data_list_train = create_data_list(train_edge_indices_bi, train_edge_indices_uni, x_train, y_train)  # Training Edges
data_list_tt = create_data_list(edge_indices['tt_bi'], edge_indices['tt_uni'], x_test, np.expand_dims(labels_tt_test, axis=1))  # True-True Edges
data_list_ll = create_data_list(edge_indices['ll_bi'], edge_indices['ll_uni'], x_test, np.expand_dims(labels_ll_test, axis=1))  # Lone-lone Edges
data_list_cl = create_data_list(edge_indices['cl_bi'], edge_indices['cl_uni'], x_test, np.expand_dims(labels_cl_test, axis=1))  # Cluster-Lone Edges
data_list_lc = create_data_list(edge_indices['lc_bi'], edge_indices['lc_uni'], x_test, np.expand_dims(labels_lc_test, axis=1))  # Lone-Cluster Edges
data_list_cc = create_data_list(edge_indices['cc_bi'], edge_indices['cc_uni'], x_test, np.expand_dims(labels_cc_test, axis=1))  # Cluster-Cluster

In [39]:
# Batch size value
batch_size = 1

# Create the data loaders
data_loader = {}
data_list_mapping = {
    "train": data_list_train,  # Training Edges
    "tt": data_list_tt,           # True-True Edges
    "ll": data_list_ll,           # Lone-Lone Edges
    "lc": data_list_lc,           # Lone-Cluster Edges
    "cl": data_list_cl,           # Cluster-Lone Edges
    "cc": data_list_cc            # Cluster-Cluster Edges
}

# Total background dataset
data_list_total_bkg = data_list_ll + data_list_cl + data_list_lc + data_list_cc
data_loader_total_bkg = torch.utils.data.DataLoader(
    custom_dataset(data_list_total_bkg),
    batch_size=batch_size,
    collate_fn=lambda batch: collate_data(batch)
)
# For the other datasets
for key, data_list in data_list_mapping.items():
    data_loader[key] = torch.utils.data.DataLoader(
        custom_dataset(data_list),
        batch_size=batch_size,
        collate_fn=lambda batch: collate_data(batch)
    )

### Choosing the Device to Run on

In [40]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Creating the weights to be used in the CrossEntropyLoss() later
The class weights will be calculated using this formula in mind (since this formula helps in the mitigation of rare-class overweighting)
$${weight}_i = \frac{{total\space samples}}{{frequency}_i}$$

In [ ]:
labels_training_flat = labels_training.flatten()

# Compute class frequencies (counts) for the entire dataset
train_label_counts = Counter(labels_training_flat)

# Total number of samples (the number of elements in the flattened array)
total_train_samples = labels_training_flat.size

# Compute class weights using the new formula
class_weights = {label: total_train_samples /
                 count for label, count in train_label_counts.items()}

# Convert weights to a tensor for PyTorch
unique_classes = np.unique(labels_training_flat)  # Get unique classes
weight_tensor = torch.tensor(
    [class_weights[label] for label in unique_classes], dtype=torch.float).to(device)

# Print class weights
print("Class Weights (Training):", class_weights)
print("Weight Tensor (Training):", weight_tensor)

### Creation of the Multi-Edge Classifier model

In [61]:
class MultiEdgeClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, layer_weights=False, debug=False):
        super(MultiEdgeClassifier, self).__init__()
        self.debug = debug
        self.layer_weights_enabled = layer_weights  # Store setting

        # Node embedding layer
        self.node_embedding = nn.Linear(input_dim, hidden_dim)

        # Initialize convolution and batch norm layers
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()
        self.layer_weights = nn.ParameterList() if layer_weights else None  # Only create if enabled

        # First layer
        self.convs.append(GCNConv(hidden_dim, 128))
        self.bns.append(BatchNorm1d(128))
        if layer_weights:
            self.layer_weights.append(nn.Parameter(torch.tensor(1.0, requires_grad=True)))

        # Additional layers
        for i in range(1, num_layers):
            in_channels = 128 if i == 1 else 64
            out_channels = 64
            self.convs.append(GCNConv(in_channels, out_channels))
            self.bns.append(BatchNorm1d(out_channels))
            if layer_weights:
                self.layer_weights.append(nn.Parameter(torch.tensor(1.0, requires_grad=True)))

        # Edge classification layer
        self.fc = nn.Linear(128, output_dim)

    def debug_print(self, message):
        if self.debug:
            print(message)

    def forward(self, x, edge_index, edge_index_out):
        # Node embedding
        x = self.node_embedding(x)
        self.debug_print(f"Node embedding output shape: {x.shape}")

        if x.dim() == 3 and x.size(0) == 1:  # Check and remove batch dimension
            x = x.squeeze(0)

        # Loop through convolution layers
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            self.debug_print(f"After GCNConv {i+1}: {x.shape}")
            if x.dim() == 3 and x.size(0) == 1:
                x = x.squeeze(0)
            x = self.bns[i](x)
            x = torch.relu(x)

            # Apply layer weight if enabled
            if self.layer_weights_enabled:
                x = x * self.layer_weights[i]
                self.debug_print(f"After Layer Weight {i+1}: {x.shape}")

        # Edge representations
        edge_rep = torch.cat([x[edge_index_out[0]], x[edge_index_out[1]]], dim=1)
        self.debug_print(f"Edge representation shape: {edge_rep.shape}")

        # Return Logits
        edge_scores = self.fc(edge_rep)
        return edge_scores


In [62]:
input_dim = 5
hidden_dim = 256
output_dim = 5  # Multiclass classification
num_layers = 6
model = MultiEdgeClassifier(input_dim, hidden_dim, output_dim, num_layers, layer_weights=True)

num_epochs = 300
criterion = nn.CrossEntropyLoss(weight=weight_tensor)
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.97)

### Creation of functions to run our model

In [57]:
def train_model(model, device, data_loader, optimizer, criterion):
    # Sets the model into training mode
    model.train()
    # Sends model to GPU if available, otherwise uses the CPU
    model.to(device)

    # Assumes there is only one batch in the data loader
    # Retrieve the single batch from the data loader
    batch_x, batch_edge_index, batch_edge_index_out, batch_y = next(
        iter(data_loader))

    # Sends the input features, the edge indices, and target
    # labels to the GPU if available, otherwise the CPU
    batch_x = torch.stack(batch_x).to(device)
    batch_edge_index = [edge_index.to(device)
                        for edge_index in batch_edge_index]
    batch_edge_index_out = [edge_index.to(
        device) for edge_index in batch_edge_index_out]

    # Convert target labels to LongTensor (torch.int64)
    batch_y = [y.long().to(device) for y in batch_y]

    # Clears the gradients of the model parameters to ensure
    # they are not accumulated across batches
    optimizer.zero_grad()

    # Initialize loss tracking for subgraphs in the single batch
    loss_per_batch = []

    # Model processes each graph in the batch one by one
    for i in range(len(batch_edge_index)):
        # Pass the features and the edge indices into the model and store
        # the output (logits)
        _output = model(batch_x[i], batch_edge_index[i],
                        batch_edge_index_out[i])

        # Ensure that model outputs (logits) are of type float32
        _output = _output.float()

        # Calculate the difference between the model output and the targets
        # via the provided criterion (loss function)
        loss = criterion(_output.squeeze(), batch_y[i].squeeze())

        # This difference is stored in the loss_per_batch list
        loss_per_batch.append(loss)

    # The average loss across all subgraphs within the single batch is calculated
    total_loss_per_batch = sum(loss_per_batch) / len(loss_per_batch)

    # Computes the loss gradients with respect to the model parameters
    total_loss_per_batch.backward()

    # Updates the model parameters using the gradients
    optimizer.step()

    # Returns the total loss for the single batch
    return total_loss_per_batch

In [58]:
def test_model(model, device, data_loader_true, data_loader_bkg_dict):
    all_scores = []
    true_labels = []

    with torch.no_grad():
        model.eval()
        model.to(device)

        # Process true edges (positive class, label 1)
        batch_x, batch_edge_index, batch_edge_index_out, batch_y = next(
            iter(data_loader_true))
        batch_x = torch.stack(batch_x).to(device)
        batch_edge_index = [edge_index.to(device)
                            for edge_index in batch_edge_index]
        batch_edge_index_out = [edge_index_out.to(
            device) for edge_index_out in batch_edge_index_out]

        for i in range(len(batch_edge_index)):
            test_edge_scores = model(
                batch_x[i], batch_edge_index[i], batch_edge_index_out[i])
            test_edge_scores = F.softmax(test_edge_scores, dim=1)

            all_scores.append(test_edge_scores)
            true_labels.append(torch.ones(test_edge_scores.size(
                0), dtype=torch.long, device=device))

        # Process background edges
        for background_type, data_loader_bkg in data_loader_bkg_dict.items():
            batch_x, batch_edge_index, batch_edge_index_out, batch_y = next(
                iter(data_loader_bkg))
            batch_x = torch.stack(batch_x).to(device)
            batch_edge_index = [edge_index.to(device)
                                for edge_index in batch_edge_index]
            batch_edge_index_out = [edge_index_out.to(
                device) for edge_index_out in batch_edge_index_out]

            for i in range(len(batch_edge_index)):
                test_edge_scores = model(
                    batch_x[i], batch_edge_index[i], batch_edge_index_out[i])
                test_edge_scores = F.softmax(test_edge_scores, dim=1)

                all_scores.append(test_edge_scores)
                true_labels.append(torch.full((test_edge_scores.size(
                    0),), background_type, dtype=torch.long, device=device))

    # Concatenate all scores and labels
    all_scores = torch.cat(all_scores, dim=0)
    true_labels = torch.cat(true_labels, dim=0)

    # Reorder scores and labels to match the desired order (0, 1, 2, 3, 4)
    desired_order = [0, 1, 2, 3, 4]
    # Ensure reordering matches desired labels
    mask = torch.argsort(true_labels).argsort()
    all_scores = all_scores[mask]
    true_labels = true_labels[mask]

    return all_scores.cpu().numpy(), true_labels.cpu().numpy()

In [59]:
def loss_for_train_and_test(model, loader, loss_fn, optimizer, training, device):
    if training:
        model.train()
    else:
        model.eval()

    total_loss = 0.0
    num_batches = 0
    all_logits = []  # Store raw logits in testing mode

    for batch in loader:
        if training:
            batch_x, batch_edge_index, batch_edge_index_out, batch_y = batch
            batch_y = batch_y.to(device)
        else:
            batch_x, batch_edge_index, batch_edge_index_out, *_ = batch

        # Move features and edge indices to the device
        batch_x = torch.stack(batch_x).to(device)
        batch_edge_index = [edge_index.to(device)
                            for edge_index in batch_edge_index]
        batch_edge_index_out = [edge_index_out.to(
            device) for edge_index_out in batch_edge_index_out]

        if training:
            for i in range(len(batch_edge_index)):
                optimizer.zero_grad()

                # Forward pass
                logits = model(
                    batch_x[i], batch_edge_index[i], batch_edge_index_out[i])

                # Compute loss
                loss = loss_fn(logits, batch_y[i])
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                num_batches += 1
        else:
            with torch.no_grad():
                for i in range(len(batch_edge_index)):
                    logits = model(
                        batch_x[i], batch_edge_index[i], batch_edge_index_out[i])
                    all_logits.append(logits)  # Store raw logits
                    num_batches += 1

    average_loss = total_loss / num_batches if training else None
    all_logits = torch.cat(all_logits, dim=0).cpu(
    ).numpy() if all_logits else None

    return average_loss, all_logits

## Running the model over num_epochs iteration

In [60]:
data_loader_bkg_dict = {
    0: data_loader['ll'],  # For label 0
    2: data_loader['cl'],  # For label 2
    3: data_loader['lc'],  # For label 3
    4: data_loader['cc']   # For label 4
}

loss_per_epoch = []
scores = []
truth_labels = []
avg_loss_training_true_class = []
logits_training_true_class = []
avg_loss_testing_true_class = []
logits_testing_true_class = []
avg_loss_training_bkg_classes = []
logits_training_bkg_classes = []
avg_loss_testing_bkg_classes = []
logits_testing_bkg_classes = []

for epoch in range(num_epochs):
    # Train the model
    total_loss_per_epoch = train_model(model, device, data_loader['train'], optimizer, criterion)
    # Ensure tensor is detached for saving
    loss_per_epoch.append(total_loss_per_epoch.cpu().detach().numpy())

    # Update learning rate
    scheduler.step()

    # Test the model
    epoch_scores, epoch_true_labels = test_model(model, device, data_loader['tt'], data_loader_bkg_dict)

    # Compute the average loss for true and background edges
    avgLossTrueTrain, logitsTrueTrain = loss_for_train_and_test(model, data_loader['train'], criterion, optimizer, True, device)
    avgLossTrueTest, logitsTrueTest = loss_for_train_and_test(model, data_loader['tt'], criterion, optimizer, False, device)
    avgLossBkgTrain, logitsBkgTrain = loss_for_train_and_test(model, data_loader['train'], criterion, optimizer, True, device)
    avgLossBkgTest, logtisBkgTest = loss_for_train_and_test(model, data_loader_total_bkg, criterion, optimizer, False, device)

    # Store results
    scores.append(epoch_scores)
    truth_labels.append(epoch_true_labels)
    avg_loss_training_true_class.append(avgLossTrueTrain)
    logits_training_true_class.append(logitsTrueTrain)
    avg_loss_testing_true_class.append(avgLossTrueTest)
    logits_testing_true_class.append(logitsTrueTest)
    avg_loss_training_bkg_classes.append(avgLossBkgTrain)
    logits_training_bkg_classes.append(logitsBkgTrain)
    avg_loss_testing_bkg_classes.append(avgLossBkgTest)
    logits_testing_bkg_classes.append(logtisBkgTest)

    # Print the loss for the current epoch
    print(f"Epoch: {epoch+1} | Total Loss Per Epoch: {total_loss_per_epoch.item():.4f}")

RuntimeError: The size of tensor a (64) must match the size of tensor b (256) at non-singleton dimension 1

In [ ]:
loss_per_epoch = np.array(loss_per_epoch)
scores = np.array(scores)
truth_labels = np.array(truth_labels)
avg_loss_training_true_class = np.array(avg_loss_training_true_class)
logits_training_true_class = np.array(logits_training_true_class)
avg_loss_testing_true_class = np.array(avg_loss_testing_true_class)
logits_testing_true_class = np.array(logits_testing_true_class)
avg_loss_training_bkg_classes = np.array(avg_loss_training_bkg_classes)
logits_training_bkg_classes = np.array(logits_training_bkg_classes)
avg_loss_testing_bkg_classes = np.array(avg_loss_testing_bkg_classes)
logits_testing_bkg_classes = np.array(logits_testing_bkg_classes)

print(loss_per_epoch.shape)
print(scores.shape)
print(truth_labels.shape)
print(avg_loss_training_true_class.shape)
print(logits_training_true_class.shape)
print(avg_loss_testing_true_class.shape)
print(logits_testing_true_class.shape)
print(avg_loss_training_bkg_classes.shape)
print(logits_training_bkg_classes.shape)
print(avg_loss_testing_bkg_classes.shape)
print(logits_testing_bkg_classes.shape)

(300,)
(300, 163000, 5)
(300, 163000)
(300,)
(300,)
(300,)
(300, 800000, 5)
(300,)
(300,)
(300,)
(300, 2460000, 5)


In [ ]:
def save_data_pickle(file_name, save_path, data_dict):
    """
    Saves the given data dictionary into a Pickle (.pkl) file.
    
    Parameters:
        file_name (str): Name of the Pickle file.
        save_path (str): Directory where the file will be saved.
        data_dict (dict): Dictionary containing data to be saved.
    """
    # Ensure the save directory exists
    os.makedirs(save_path, exist_ok=True)

    # Full file path
    full_path = os.path.join(save_path, file_name)

    with open(full_path, 'wb') as file:
        pickle.dump(data_dict, file, protocol=pickle.HIGHEST_PROTOCOL)

    print(f"Data successfully saved to {full_path}")

# Create the dictionary
data_dict = {
    "loss_per_epoch": loss_per_epoch,
    "scores": scores,
    "truth_labels": truth_labels,
    "avg_loss_training_true_class": avg_loss_training_true_class,
    "logits_training_true_class": logits_training_true_class,
    "avg_loss_testing_true_class": avg_loss_testing_true_class,
    "logits_testing_true_class": logits_testing_true_class,
    "avg_loss_training_bkg_classes": avg_loss_training_bkg_classes,
    "logits_training_bkg_classes": logits_training_bkg_classes,
    "avg_loss_testing_bkg_classes": avg_loss_testing_bkg_classes,
    "logits_testing_bkg_classes": logits_testing_bkg_classes
}

# Save the file
save_data_pickle('model_6layers_16heads_hybrid_GCNConvGAT_residual.pkl', "/storage/mxg1065", data_dict)

Data successfully saved to /storage/mxg1065/model_6layers_seed_42_a.pkl
